**Importing Libraries**

In [13]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
%matplotlib inline
# For Sentiment Analysis
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


**Store Data, Time and Headlines**

In [14]:
n = 3

stock = input("Enter the stock: ")
print(stock)
stock = stock.upper()
print(stock)
tickers=[]
tickers.append(stock)
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    result = urlopen(req)    
    # Place the contents of the file in html page named 'html'
    html = BeautifulSoup(result)
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

aapl
AAPL


**Print the above column(just for debugging purposes)**

In [15]:
try:
  for ticker in tickers:
    df = news_tables[ticker]
    df_tr = df.findAll('tr')
    for i, table_row in enumerate(df_tr):
      # Read the text of the element 'a' 
      a_text = table_row.a.text
      # Read the text of the element 'td' 
      td_text = table_row.td.text
      #Clean data using strip()
      td_text = td_text.strip()
      print(a_text)
      print(td_text)

      if i == n-1:
        break
except KeyError:
  pass

Stocks trade mixed at the open as investors await Fed rate hike decision
Mar-22-23 10:02AM
$10,000 Invested In These Growth Stocks Could Make You a Fortune Over the Next 10 Years
09:45AM
Is Now the Time to Buy Solana?
08:50AM


**Make list of data**

In [16]:
parsed_news = []
for file_name, news_table in news_tables.items():
    try:
      for x in news_table.findAll('tr'):
          text = x.a.get_text()
          date_scrape = x.td.text.split()
          
          if len(date_scrape) == 1:
            time = date_scrape[0]
              
          # else load 'date' as the 1st element and 'time' as the second    
          else:
            date = date_scrape[0]
            time = date_scrape[1]
          # Extract the ticker from the file name, get the string up to the 1st '_'  
          ticker = file_name.split('_')[0]
          
          parsed_news.append([ticker, date, time, text])
      
    except KeyError:
      pass    
parsed_news

[['AAPL',
  'Mar-22-23',
  '10:02AM',
  'Stocks trade mixed at the open as investors await Fed rate hike decision'],
 ['AAPL',
  'Mar-22-23',
  '09:45AM',
  '$10,000 Invested In These Growth Stocks Could Make You a Fortune Over the Next 10 Years'],
 ['AAPL', 'Mar-22-23', '08:50AM', 'Is Now the Time to Buy Solana?'],
 ['AAPL',
  'Mar-22-23',
  '06:00AM',
  '3 things to watch in TikTok CEO testimony to Congress'],
 ['AAPL',
  'Mar-22-23',
  '05:45AM',
  'Nasdaq Bear Market: 1 Key Reason Apple Has Avoided Major Job Cuts'],
 ['AAPL',
  'Mar-22-23',
  '05:30AM',
  'Apple, Microsoft Dominate U.S. Markets After FAANG Trade Fizzles'],
 ['AAPL',
  'Mar-22-23',
  '05:20AM',
  '3 Blue-Chip Dividend Stocks That Generate Gobs of Shareholder Value'],
 ['AAPL', 'Mar-21-23', '08:00PM', 'Apple Lobbies India Hard on Labor Law'],
 ['AAPL', 'Mar-21-23', '02:08PM', 'Apple Stock Gears Up for Major Breakout'],
 ['AAPL',
  'Mar-21-23',
  '01:06PM',
  "Apple's Latest Moves To Abandon China Require A Big Lift: 

**Apply Sentiment Analysis**

In [17]:
vader_analysis = SentimentIntensityAnalyzer()
columns = ['Ticker', 'Date', 'Time', 'Headline']

parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

scores = parsed_and_scored_news['Headline'].apply(vader_analysis.polarity_scores).tolist()

scores_df = pd.DataFrame(scores)

parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

parsed_and_scored_news['Date'] = pd.to_datetime(parsed_and_scored_news.Date).dt.date


parsed_and_scored_news = parsed_and_scored_news.set_index('Ticker')

parsed_and_scored_news


,Date,Time,Headline,neg,neu,pos,compound
Ticker,,,,,,,
AAPL,2023-03-22,10:02AM,Stocks trade mixed at the open as investors aw...,0.000,0.896,0.104,0.1027
AAPL,2023-03-22,09:45AM,"$10,000 Invested In These Growth Stocks Could ...",0.000,0.843,0.157,0.3818
AAPL,2023-03-22,08:50AM,Is Now the Time to Buy Solana?,0.000,1.000,0.000,0.0000
AAPL,2023-03-22,06:00AM,3 things to watch in TikTok CEO testimony to C...,0.000,1.000,0.000,0.0000
AAPL,2023-03-22,05:45AM,Nasdaq Bear Market: 1 Key Reason Apple Has Avo...,0.338,0.662,0.000,-0.5574
...,...,...,...,...,...,...,...
AAPL,2023-03-14,04:07PM,California court upholds Prop. 22 ruling on gi...,0.000,0.753,0.247,0.5574
AAPL,2023-03-14,02:45PM,Apple plans to delay bonuses for some employee...,0.178,0.543,0.279,0.3182
AAPL,2023-03-14,02:11PM,Silicon Valley Bank fallout likely to hit big ...,0.000,1.000,0.000,0.0000


**Display the Final Result**

In [18]:
df = parsed_and_scored_news
df = df.drop(columns = ['Headline','Date','Time', 'neg', 'pos', 'neu'])
mean = round(df.mean(), 4)
print(df)
print("Final Sentiment: ", mean)
# a positive sentiment, compound ≥ 0.05
# a negative sentiment, compound ≤ -0.05
# a neutral sentiment, the compound is between [-0.05, 0.05]

        compound
Ticker          
AAPL      0.1027
AAPL      0.3818
AAPL      0.0000
AAPL      0.0000
AAPL     -0.5574
...          ...
AAPL      0.5574
AAPL      0.3182
AAPL      0.0000
AAPL      0.5574
AAPL      0.0000

[100 rows x 1 columns]
Final Sentiment:  compound    0.1229
dtype: float64
